In [138]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from scipy import stats
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from math import log,exp
from sklearn import base

In [74]:
test = pd.read_csv("/content/data_test.csv",header=None)
train = pd.read_csv("/content/data_train.csv",header=None)

In [ ]:
print(test.shape)
print(train.shape)

(3498, 17)
(7494, 17)


## الف)
Random Forest

In [ ]:
train1=train.copy()
test1=test.copy()
train1=train1.to_numpy()
test1=test1.to_numpy()

In [ ]:
print(test1.shape)

(3498, 17)


bootstrap

In [ ]:
#function returns a random sample with size m from dataframe (with replacement) in the form of data and taget
def bootstrapSample(dataFrame):
    randomIndices = np.random.randint(low = 0, high = len(dataFrame), size = len(dataFrame))
    x=dataFrame[randomIndices]
    data=x[:, :-1]
    target=x[:,-1]
    return data,target

aggregation

In [ ]:
#seperating data and labels in test set
test_data=test1[: , :-1]
test_label=test1[: , -1]

#possible classes are 0,...,9
possible_classes = list(dict.fromkeys(train1[:,-1]))

#numpy array to store the result of all classifiers
result=np.zeros((15,len(test_label)))
for i in range(15):
  train_data , train_label=bootstrapSample(train1)
  decision_tree = DecisionTreeClassifier(max_depth=3,max_features=3)
  decision_tree = decision_tree.fit(train_data, train_label)
  #prediction result for each bootstrap data
  result[i]=decision_tree.predict(test_data)
#result for each data is the mode of all classifier's results
predicted_label=stats.mode(result)[0]
predicted_label=predicted_label.reshape(len(test_label))
print("accuracy: "+str(accuracy_score(y_true=test_label,y_pred=predicted_label)))
print(confusion_matrix(y_true=test_label, y_pred=predicted_label, labels=range(len(possible_classes))))

accuracy: 0.6460834762721556
[[324   2   6   0   0   0  31   0   0   0]
 [  0 151 206   3   1   0   3   0   0   0]
 [  0   4 343   2   0   0  13   2   0   0]
 [  0   7   1 327   1   0   0   0   0   0]
 [  0   5   0   0 343   0  16   0   0   0]
 [  2   7   9 118  30 166   3   0   0   0]
 [  2   0  15   2   6   0 306   5   0   0]
 [  1  54  14   2   0   2   0 291   0   0]
 [122   0  14   0   0  76  46  69   9   0]
 [  1  69   5 190  69   0   2   0   0   0]]


## ب)
AdaBoost

In [75]:
train2=train.copy()
test2=test.copy()
train2=train2.to_numpy()
test2=test2.to_numpy()

In [111]:
a=np.array([1,2,3,4])
a_pred=np.array([1,2,5,4])
print((np.not_equal(a, a_pred)).astype(int) )


[0 0 1 0]


In [193]:
def ada_boost_fit(X,y,num):
  #X is the train data
  #y is the train label
  #num is the number of trees
    classifiers=[]
    alphas=[]
    err=[]
    N = len(y)
    #initial w
    w = np.array([1/N for i in range(N)])
    #number of classes
    k =10
        
    for m in range(num):
            
      Gm = base.clone(DecisionTreeClassifier(max_depth = 1)).\
                            fit(X,y,sample_weight=w).predict
      classifiers.append(Gm)
            
      incorrect = Gm(X) != y
      errM = np.average(incorrect,weights=w,axis=0)
            
      err.append(errM)

      #for multiple classification      
      BetaM = (np.log((1-errM)/errM)+np.log(k-1))
      alphas.append(BetaM)

      #updating w   
      w *= np.exp(BetaM*incorrect*(w > 0))

    return classifiers,err,alphas
 


In [6]:
#possible_classes = list(dict.fromkeys(train1[:,-1]))
possible_classes=[0,1,2,3,4,5,6,7,8,9]
print(possible_classes)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [142]:
#function chooses the class thay has the highest betha in one row(could be a summation of a number of bethas)
def voting(row):
  probablity=np.zeros(len(possible_classes))
  for p in range(len(possible_classes)):
    for r in range(len(row)):
      if row[r][0]==p:
        probablity[p]=probablity[p]+row[r][1]
  indices = np.where(probablity == probablity.max())
  return indices[0][0]



In [182]:
row=[(7,0.6),(3,0.5),(3,0.3)]
print(voting(row))

3


In [183]:
#predict using fitted model
def predict(test_data,num,classifiers,alphas):
  y_pred=[]
  alphas=alphas/sum(alphas)
  # Initialise dataframe with weak predictions for each observation
  weak_preds = pd.DataFrame(index = range(len(test_data)), columns = range(num))
  

  # Predict class label for each weak classifier, weighted by alpha_m
  for m in range(num):
    y_pred_m=[]
    ##print(classifiers[m])
    #classifier m's result
    tmp=classifiers[m](test_data)
    ##print(tmp)
    for i in range(len(test_data)):
      #for the ith labe: classifier m's result and the related weight
      y_pred_m.append((tmp[i],alphas[m]))
    #print(y_pred_m)
    weak_preds.iloc[:,m] = y_pred_m
  #print(weak_preds)

  # Estimate final predictions
  for i in range(len(test_data)):
    #print(weak_preds.iloc[i,:])
    #find the class with the most weight
    y_pred.append(voting(weak_preds.iloc[i,:]))
  return y_pred

In [171]:
#Get the error rates of each weak classifier.
def error_rates(num, data, label,classifiers):
  prediction_errors = []  
  # Predict class label for each weak classifier
  for m in range(num):
    y_pred_m = classifiers[m](data)          
    error_m = compute_error(label,y_pred_m, w_i = np.ones(len(label)))
    prediction_errors.append(error_m)
  return prediction_errors
          

In [194]:
clf,training_err,alphas=ada_boost_fit(train2[:,:-1],train2[:,-1],10)
y_pred=predict(test2[:,:-1],10,clf,alphas)
print("accuracy: "+str(accuracy_score(y_true=test2[:,-1],y_pred=y_pred)))
##print(error_rates(10,train2[:,:-1],train2[:,-1],clf))

accuracy: 0.49342481417953116


## ج)

number of trees=5

In [187]:
clf,training_err,alphas=ada_boost_fit(train2[:,:-1],train2[:,-1],5)
y_pred=predict(test2[:,:-1],5,clf,alphas)
print("accuracy: "+str(accuracy_score(y_true=test2[:,-1],y_pred=y_pred)))


accuracy: 0.15208690680388792


number of trees=20

In [188]:
clf,training_err,alphas=ada_boost_fit(train2[:,:-1],train2[:,-1],20)
y_pred=predict(test2[:,:-1],20,clf,alphas)
print("accuracy: "+str(accuracy_score(y_true=test2[:,-1],y_pred=y_pred)))


accuracy: 0.44882790165809033


number of trees=50

In [189]:
clf,training_err,alphas=ada_boost_fit(train2[:,:-1],train2[:,-1],50)
y_pred=predict(test2[:,:-1],50,clf,alphas)
print("accuracy: "+str(accuracy_score(y_true=test2[:,-1],y_pred=y_pred)))


accuracy: 0.5877644368210406
